1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import math
import re
import warnings

from pymongo import MongoClient

warnings.filterwarnings("ignore")

In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}

In [3]:
client = MongoClient('127.0.0.1', 27017)
db = client['my_database']
vac = db.vacancies

In [4]:
def hh_search():
    
    base_hh_url = 'https://hh.ru'
    
    for i in range(pages):
        full_hh_url = base_hh_url + f'/search/vacancy?area=1&fromSearchLine=true&text={search_query}&from=suggest_post&page={i}&hhtmFrom=vacancy_search_list'
        response = requests.get(full_hh_url, headers = headers)
        with open('response.html', 'w', encoding = 'utf-8') as f:
            f.write(response.text)
        html = ''
        with open('response.html', 'r', encoding = 'utf-8') as f:
            html = f.read()
        dom = bs(html, 'html.parser')
        vacancies = dom.find_all('div', {'class': "vacancy-serp-item-body__main-info"})

        for vacancy in vacancies:
            vacancy_data = {}
            vacancy_link = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title' })['href']
            vacancy_name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title' }).getText()
            site = base_hh_url
            sallary = vacancy.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'})

            if sallary:
                text = sallary.getText()

                if 'от' in text:

                    min_sallary = re.findall('[0-9]+', text)
                    min_sallary = int(''.join(min_sallary))
                    max_sallary = None
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]                   
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = site
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)

                    continue
                
                if 'до' in text:

                    min_sallary = None
                    max_sallary = re.findall('[0-9]+', text)
                    max_sallary = int(''.join(max_sallary))
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = site
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)
                    continue
                
                else:
                    sallary_ = re.findall('[0-9]+', text)
                    min_sallary = int(sallary_[0] + sallary_[1])
                    max_sallary = int(sallary_[2] + sallary_[3])
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[0]
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = site
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)
                    continue
            
            else:
                min_sallary = None
                max_sallary = None
                currency = None
                vacancy_data['link'] = vacancy_link
                vacancy_data['name'] = vacancy_name
                vacancy_data['min'] = min_sallary
                vacancy_data['max'] = max_sallary
                vacancy_data['currency'] = currency
                vacancy_data['site'] = site
                if len(list(vac.find({'link': vacancy_link}))) == 0:
                    vac.insert_one(vacancy_data)

In [5]:
def sj_search():
    base_sj_url = 'https://russia.superjob.ru'
    for i in range(pages):
        full_sj_url = base_sj_url + f'/vacancy/search/?keywords={search_query}&page={i}'
        response = requests.get(full_sj_url, headers = headers, verify=False)
        with open('response.html', 'w', encoding = 'utf-8') as f:
            f.write(response.text)
        html = ''
        with open('response.html', 'r', encoding = 'utf-8') as f:
            html = f.read()
        dom = bs(html, 'html.parser')
        vacancies = dom.find_all('div', {'class': '_3JPvf _1O2dw PEEVg WuWPa'})

        for vacancy in vacancies:
            vacancy_data = {}
            vacancy_link = base_sj_url + vacancy.find('a', {'class': '_1IHWd'})['href']
            vacancy_name = vacancy.find('span').getText()

            sallary = vacancy.find('span', {'class' : '_2eYAG'})
            if sallary:
                text = sallary.getText()

                if 'По договорённости' in text:
                    min_sallary = None
                    max_sallary = None
                    currency = None
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = base_sj_url
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)
                        
                    continue


                if 'от' in text:

                    min_sallary = re.findall('[0-9]+', text)
                    min_sallary = int(''.join(min_sallary))
                    max_sallary = None
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = base_sj_url
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)

                    continue

                if 'до' in text:

                    min_sallary = None
                    max_sallary = re.findall('[0-9]+', text)
                    max_sallary = ''.join(max_sallary)
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    vacancy_data['link'] = vacancy_link
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['min'] = min_sallary
                    vacancy_data['max'] = max_sallary
                    vacancy_data['currency'] = currency
                    vacancy_data['site'] = base_sj_url
                    if len(list(vac.find({'link': vacancy_link}))) == 0:
                        vac.insert_one(vacancy_data)

                    continue

                else:

                    sallary_ = re.findall('[0-9]+', text)
                    if len(sallary_) > 2:
                        min_sallary = int(sallary_[0] + sallary_[1])
                        max_sallary = int(sallary_[2] + sallary_[3])
                        cur_sallary = re.findall('[A-zА-я]+', text)
                        currency = cur_sallary[0]
                        vacancy_data['link'] = vacancy_link
                        vacancy_data['name'] = vacancy_name
                        vacancy_data['min'] = min_sallary
                        vacancy_data['max'] = max_sallary
                        vacancy_data['currency'] = currency
                        vacancy_data['site'] = base_sj_url
                        if len(list(vac.find({'link': vacancy_link}))) == 0:
                            vac.insert_one(vacancy_data)
                    continue
                        
            else:
                min_sallary = int(sallary_[0] + sallary_[1])
                max_sallary = int(sallary_[0] + sallary_[1])
                cur_sallary = re.findall('[A-zА-я]+', text)
                currency = cur_sallary[0]
                vacancy_data['link'] = vacancy_link
                vacancy_data['name'] = vacancy_name
                vacancy_data['min'] = min_sallary
                vacancy_data['max'] = max_sallary
                vacancy_data['currency'] = currency
                vacancy_data['site'] = base_sj_url
                if len(list(vac.find({'link': vacancy_link}))) == 0:
                    vac.insert_one(vacancy_data)

In [6]:
def get_result():
    hh_search()
    sj_search()   

In [7]:
search_query = input('Введите название вакансии для поиска: ').replace(' ', '+')
pages = int(input('Введите количество страниц сайта, по которым необходимо собрать информацию: '))

Введите название вакансии для поиска: хирург
Введите количество страниц сайта, по которым необходимо собрать информацию: 3


In [8]:
get_result()
cnt = list(vac.find({}))
print(f'Кол-во записей после первой вставки: {len(cnt)}')

get_result()
cnt = list(vac.find({}))
print(f'Кол-во записей после второй вставки: {len(cnt)}')

get_result()
cnt = list(vac.find({}))
print(f'Кол-во записей после третьей вставки: {len(cnt)}')

Кол-во записей после первой вставки: 100
Кол-во записей после второй вставки: 100
Кол-во записей после третьей вставки: 100


In [9]:
#vac.delete_many({})

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [58]:

def your_vacancies():
    stop = False
    try:
        input_ = int(input('Введите размер зарплаты для отбора вакансий: '))        
    except ValueError:
        print('Зарплата может быть только числом, причем положительным!')
        stop = True
    
    if stop:
        pass
    else:
        cnt = 0
        list_ = []
        for doc in vac.find({}):
            if doc['max'] == None:
                if doc['min'] == None:
                    continue
                if doc['min'] > input_:
                    cnt += 1
                    list_.append(doc)
                    continue
                else:
                    continue
            elif doc['max'] > input_:
                cnt += 1
                list_.append(doc)
                continue
            else:
                continue
        if len(list_) > 0:
            df = pd.DataFrame(data = list_)
            display(df)
        if cnt == 0:
            print('А что ты умеешь такого, что просишь такую зарплату?')            

In [61]:
your_vacancies()

Введите размер зарплаты для отбора вакансий: 500000
А что ты умеешь такого, что просишь такую зарплату?


In [62]:
your_vacancies()

Введите размер зарплаты для отбора вакансий: 300000


,_id,link,name,min,max,currency,site
0,624f5e74ae23331dd1e853f1,https://hh.ru/vacancy/54606212?query=%D1%85%D0...,Стоматолог-хирург,250000,500000.0,руб,https://hh.ru
1,624f5e74ae23331dd1e853f3,https://hh.ru/vacancy/54636500?query=%D1%85%D0...,Стоматолог универсал (ортопед / терапевт / хир...,250000,400000.0,руб,https://hh.ru
2,624f5e74ae23331dd1e853fe,https://hh.ru/vacancy/54454534?query=%D1%85%D0...,Врач стоматолог-универсал,350000,NaN,руб,https://hh.ru
3,624f5e74ae23331dd1e8541e,https://hh.ru/vacancy/53265647?query=%D1%85%D0...,Врач-косметолог,270000,350000.0,руб,https://hh.ru


In [63]:
your_vacancies()

Введите размер зарплаты для отбора вакансий: до х..я
Зарплата может быть только числом, причем положительным!
